<a href="https://colab.research.google.com/github/finr/data-analysis-class/blob/master/Dask_Dataframes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dask Dataframes


*   Uma alternativa ao Pandas dataframe que permite paralelizar operações.
*   Um Dask DataFrame é um grande **DataFrame paralelo** composto por muitos DataFrames menores do Pandas.

*   O **tamanho da partição** desempenha um papel importante na determinação da aceleração que pode ser obtida durante a execução de cálculos.
*   Permite a execução de **cálculos paralelos** em Dataframes utilizando **múltiplos cores**.
*   **API** similar ao Pandas (com pequenas exceções).
*   Permite o **carregamento** de Datasets **maiores do que a memória**.
*   Pode ser **escalado** para **clusters**.
*   Referência: https://docs.dask.org/en/latest/dataframe.html



In [0]:
from s3fs import S3FileSystem

s3 = S3FileSystem(anon=True)
s3.ls('cesarschool-data-samples/tennis/')

In [0]:
import dask.dataframe as dd

df = dd.read_csv('s3://cesarschool-data-samples/tennis/atp_matches_201*.csv', storage_options={'anon': True}) 

In [0]:
df

In [0]:
df.describe().visualize() #.visualize(filename='describe.png')

In [0]:
df.head()

In [0]:
max_ht_country = df.groupby('winner_ioc')['winner_ht'].max()

In [0]:
max_ht_country.visualize()

In [0]:
max_ht_country.compute()

In [0]:
df.nlargest(3, 'winner_ht').compute()

In [0]:
without_cro = df[df.winner_ioc != 'CRO']

In [0]:
without_cro.nlargest(3, 'winner_ht').compute()

In [0]:
df['tourney_date'].min().compute()

In [0]:
df.dtypes

In [0]:
from datetime import datetime

def get_date_from_int (date, *args, **kwds):
  return datetime.strptime(str(date), '%Y%m%d')

df_dummy = df['tourney_date'].apply(get_date_from_int, axis=1, meta=datetime)

In [0]:
df_dummy

In [0]:
df_dummy.visualize()

In [0]:
df_dummy.compute()

In [0]:
df['year'] = df.apply(lambda x: datetime.strptime(str(x.loc['tourney_date']), '%Y%m%d').year, axis=1, meta=int)

In [0]:
df.head()

In [0]:
games_per_year = df.groupby('year')['tourney_date'].count()

In [0]:
games_per_year.visualize()

In [0]:
games_per_year.compute()

In [0]:
games_per_year.compute().plot()